ALTERNATIVA 1) Sistema de recomendación

La idea consiste en crear un modelo de recomendacion. Pueden descargar los archivos desde el siguiente link:

https://drive.google.com/drive/folders/1F58VdEmIUakN7dOvaLeEmd5xCRrSJgB6?usp=sharing


Consigna:

- El primer paso consiste en lograr leer los archivos desde el drive y armar los 2 datasets, el de entrenamiento y el de validacion.

- Se pide ademas, crear un 3er conjunto de testeo donde deberan extraer de los primeros 2 un sample y no utilizar estos datos para entrenar. Es decir, del conjunto total de datos, armar 3 dfs, training, test y validation.

- Desarrollar un recomendador. El recomendador debe ser capaz de generar recomendaciones para TODOS los usuarios (incluyendo los cold start que no tengan visualizaciones en el set de train). Generar 20 recomendaciones por usuario.
- Las recomendaciones tienen que ser para cada account_id y hay que recomendar content_id.
- Los contenidos que recomienden, no tienen que haber sido vistos previamente por los usuarios (filtrar). Se acepta solamente que se recomiende este caso si el usuario vio el contenido solamente 1 vez y el mismo posee un rating que se encuentre en el ultimo decil.
- Evaluarlo con MAP.
- Se valora la originalidad en el codigo.


Recomendaciones:
- Al momento de leer los datos, los archivos llamados "base" forman el conjunto de training y los "test" el de test.
- Quedarse con las columnas user_id; item_id; raiting; ts
- Comenzar con algo simple
- Cuando el modelo ya este funcionando, mejorarlo e ir cubriendo los distintos requisitos.
- También pueden probar con un dataframe mayor: https://grouplens.org/datasets/movielens/



---



ALTERNATIVA 2) Redes Neuronales

Aqui la eleccion del conjunto de datos es parte del desafio. Como opciones se sugiere:

1.  Una red para poder clasificar observaciones (tal cual vimos en primer lugar).
2.  Utilizar imagenes como entrenamiento para predecir objetos o numeros en un conjunto desconocido (separar en training y test). En clase discutimos el tema de las patentes como una idea. Aca deberian aplicar la etapa de convolucion seguramente para dividir el dataset. Tambien podria ser analizar imagenes mediante filtros para detectar bordes, formas, etc.
3. Intentar predecir el valor de una accion o determinado bien mediante el uso de redes recurrentes.





---



ALTERNATIVA 3) Clustering

La idea principal es que primero consigan el dataset con el cual deberán trabajar. Una vez que tengan su dataset, la idea es que apliquen como mínimo los 2 modelos de clustering que vimos, el de K-Means y DBScan. Es necesario que incluyan una optimizacion de hiperparametros (según las metricas de Inercia y Silueta). Cuando ya tengan segmentados los datos, tienen que analizar y encontrar las caracteristicas de cada uno.



---



Fecha de entrega y exposición: 04/02 - 00 Hs. | 06/02 - 19 Hs.

Como desafio adicional, e**s necesario que suban el archivo a un repositorio de github (publico)** y enviar el link del mismo al siguiente mail: hugofolonier@gmail.com o por mensaje privado en Slack.



---



**ÍCARO - DATA SCIENCE 0423** | **ENTREGABLE 3 - ALTERNATIVAS**

**Integrantes Grupo 7:** Brown, Ignacio - Fedele, Delicia - Santos, Martín



---



###SECCIÓN I: BIBLIOTECAS Y LIBRERÍAS.

In [45]:
pip install numpy scipy

In [46]:
pip install scikit-learn

In [47]:
pip install pandas

In [48]:
import gdown
import pandas as pd
import os
import sklearn
import numpy as np
import sys
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from datetime import datetime
from scipy.sparse import csr_matrix
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.data import Dataset
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k, reciprocal_rank
from sklearn.metrics import average_precision_score
from surprise import Dataset, Reader, SVD
from surprise import accuracy
from collections import defaultdict

###SECCIÓN II: SISTEMA DE RECOMENDACIÓN.

In [49]:
#Punto 1): El primer paso consiste en lograr leer los archivos desde el drive y armar los 2 datasets, el de entrenamiento y el de validación.

In [50]:
#carpeta_id = "C:\\Users\\Crix\Downloads\\DELICIA_DS\\ml-25m"

In [51]:
#dataframes = []

In [52]:
from google.colab import drive
drive.mount('/content/drive')
carpeta_id = '/content/drive/MyDrive/FORMACIONCONTINUA/DATA SCIENCE/Colab Notebooks/GitHub/ml-25m'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
if os.path.exists(carpeta_id):
    lista_archivos = os.listdir(carpeta_id)

In [54]:
archivos_csv = [archivo for archivo in lista_archivos if archivo.endswith('.csv')]

In [55]:
for archivo_csv in archivos_csv:
    ruta_completa = os.path.join(carpeta_id, archivo_csv)

In [56]:
datasets = {}

In [57]:
#Originalmente desde la web https://grouplens.org/datasets/movielens/, descargamos de las recomendaciones propuestas el DataSet MovieLens 25M.

#MovieLens 25M movie ratings. Stable benchmark dataset. 25 million ratings and one million tag applications applied to 62,000 movies by 162,000 users.
#Includes tag genome data with 15 million relevance scores across 1,129 tags. Released 12/2019.

#README.txt
#ml-25m.zip (size: 250 MB, checksum)

In [58]:
#La carpeta contiene 6 archivos *.csv y 1 archivo *.txt
#Se hizo un trabajo previo de mergeado en una notebook aparte, para trabajar con un solo DataSet, al cual llamamos "movie1.csv" a la versión final.

In [59]:
merged_dfe = pd.read_csv(os.path.join(carpeta_id, 'ratings.csv'))

In [60]:
merged_dfe['timestamp'] = pd.to_datetime(merged_dfe['timestamp'], unit='s')

In [61]:
merged_dfe['timestamp_n'] = merged_dfe['timestamp'].dt.strftime('%Y-%m-%d')

In [62]:
merged_dfe.drop(columns=['timestamp'], inplace=True)

In [63]:
merged_dfe

,userId,movieId,rating,timestamp_n
0,1,296,5.0,2006-05-17
1,1,306,3.5,2006-05-17
2,1,307,5.0,2006-05-17
3,1,665,5.0,2006-05-17
4,1,899,3.5,2006-05-17
...,...,...,...,...
25000090,162541,50872,4.5,2009-04-28
25000091,162541,55768,2.5,2009-04-28
25000092,162541,56176,2.0,2009-04-28
25000093,162541,58559,4.0,2009-04-28


In [64]:
merged_dfe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   userId       int64  
 1   movieId      int64  
 2   rating       float64
 3   timestamp_n  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 762.9+ MB


In [65]:
nulos = merged_dfe.isnull().any()
nulos
cant = merged_dfe.isnull().sum()
cant

userId         0
movieId        0
rating         0
timestamp_n    0
dtype: int64

In [66]:
merged_dfe['timestamp_n'] = pd.to_datetime(merged_dfe['timestamp_n'])

# Ordenar el DataFrame por la columna de timestamp
df_sorted = merged_dfe.sort_values(by='timestamp_n')

# Dividir el DataFrame en diferentes grupos según los timestamps
groups = df_sorted.groupby(pd.Grouper(key='timestamp_n', freq='M'))  # Dividir por mes, puedes cambiar 'M' por otra frecuencia

# Iterar sobre cada grupo
for timestamp, group in groups:
    # Guardar el grupo en un archivo CSV separado
    filename = f"rating_{timestamp.strftime('%Y-%m')}.csv"  # Nombre del archivo basado en el timestamp
    group.to_csv(filename, index=False)  # Guardar el grupo en un archivo CSV sin incluir el índice
    print(f"Guardado {filename}")

Guardado rating_1995-01.csv
Guardado rating_1995-02.csv
Guardado rating_1995-03.csv
Guardado rating_1995-04.csv
Guardado rating_1995-05.csv
Guardado rating_1995-06.csv
Guardado rating_1995-07.csv
Guardado rating_1995-08.csv
Guardado rating_1995-09.csv
Guardado rating_1995-10.csv
Guardado rating_1995-11.csv
Guardado rating_1995-12.csv
Guardado rating_1996-01.csv
Guardado rating_1996-02.csv
Guardado rating_1996-03.csv
Guardado rating_1996-04.csv
Guardado rating_1996-05.csv
Guardado rating_1996-06.csv
Guardado rating_1996-07.csv
Guardado rating_1996-08.csv
Guardado rating_1996-09.csv
Guardado rating_1996-10.csv
Guardado rating_1996-11.csv
Guardado rating_1996-12.csv
Guardado rating_1997-01.csv
Guardado rating_1997-02.csv
Guardado rating_1997-03.csv
Guardado rating_1997-04.csv
Guardado rating_1997-05.csv
Guardado rating_1997-06.csv
Guardado rating_1997-07.csv
Guardado rating_1997-08.csv
Guardado rating_1997-09.csv
Guardado rating_1997-10.csv
Guardado rating_1997-11.csv
Guardado rating_1997

In [67]:
#A partir de aquí se arman los 2 DataSets, el de entrenamiento y el de validación.

In [68]:
#Se define el rango de calificaciones, se carga el conjunto de datos, y luego se divide en conjuntos de entrenamiento (trainset) y validación
#(validationset).

In [69]:
trainset, validationset = train_test_split(merged_dfe, test_size=0.2, random_state=42)

# Guardar los conjuntos de datos en archivos CSV
trainset.to_csv("trainset.csv", index=False)
validationset.to_csv("validationset.csv", index=False)

# Imprimir la información sobre los conjuntos de datos
print("Tamaño del conjunto de entrenamiento:", len(trainset))
print("Tamaño del conjunto de validación:", len(validationset))

Tamaño del conjunto de entrenamiento: 20000076
Tamaño del conjunto de validación: 5000019


In [70]:
#Punto 2): Se pide ademas, crear un 3er conjunto de testeo donde deberan extraer de los primeros 2 un sample y no utilizar estos datos para entrenar.
#          Es decir, del conjunto total de datos, armar 3 dfs, training, test y validation.

In [71]:
trainset, testset = train_test_split(merged_dfe, train_size=0.8, random_state=42)

# Guardar los conjuntos de datos en archivos CSV
trainset.to_csv("trainset.csv", index=False)
testset.to_csv("testset.csv", index=False)

# Imprimir la información sobre los conjuntos de datos
print("Tamaño del conjunto de entrenamiento:", len(trainset))
print("Tamaño del conjunto de prueba:", len(testset))

Tamaño del conjunto de entrenamiento: 20000076
Tamaño del conjunto de prueba: 5000019


In [72]:
#Punto 3 y 4): Desarrollar un recomendador. El recomendador debe ser capaz de generar recomendaciones para TODOS los usuarios (incluyendo los cold start
#              que no tengan visualizaciones en el set de train). Generar 20 recomendaciones por usuario.
#              Las recomendaciones tienen que ser para cada userId y hay que recomendar movieId.

In [73]:
data_files = ["rating_2019-01.csv", "rating_2019-02.csv"]  # Lista de archivos CSV correspondientes a diferentes meses
dfs = []
for file in data_files:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenar los DataFrames en uno solo
merged_dfe = pd.concat(dfs, ignore_index=True)

# Calcular la popularidad de las películas en cada mes
popularity_per_month = merged_dfe.groupby('movieId').size().reset_index(name='popularity')

# Ordenar las películas por popularidad en cada mes
popularity_per_month = popularity_per_month.sort_values(by=['popularity'], ascending=False)

# Recomendar las películas más populares del último mes
top_movies_last_month = popularity_per_month.head(10)['movieId'].tolist()

print("Top 10 películas del último mes:")
print(top_movies_last_month)

Top 10 películas del último mes:
[79132, 2571, 318, 122912, 58559, 2959, 7153, 4993, 5952, 109487]


In [74]:
data_files = ["rating_2019-01.csv", "rating_2019-02.csv"]  # Lista de archivos CSV correspondientes a diferentes períodos de tiempo
dfs = []
for file in data_files:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenar los DataFrames en uno solo
merged_dfe = pd.concat(dfs, ignore_index=True)

# Calcular la popularidad de las películas en cada período de tiempo
popularity_per_month = merged_dfe.groupby('movieId').size().reset_index(name='popularity')

# Ordenar las películas por popularidad en cada período de tiempo
popularity_per_month = popularity_per_month.sort_values(by=['popularity'], ascending=False)

# Recomendar las películas más populares del último período de tiempo para cada usuario
recommendations = {}
for user_id in merged_dfe['userId'].unique():
    # Filtrar las interacciones del usuario en el conjunto de datos
    user_data = merged_dfe[merged_dfe['userId'] == user_id]

    # Obtener el último período de tiempo en el que el usuario ha interactuado
    last_timestamp = user_data['timestamp_n'].max()

    # Filtrar el conjunto de datos para obtener solo las interacciones del último período de tiempo
    last_month_data = user_data[user_data['timestamp_n'] == last_timestamp]

    # Calcular la popularidad de las películas en el último período de tiempo
    popularity_last_month = last_month_data.groupby('movieId').size().reset_index(name='popularity')

    # Ordenar las películas por popularidad en el último período de tiempo
    popularity_last_month = popularity_last_month.sort_values(by=['popularity'], ascending=False)

    # Recomendar las películas más populares del último período de tiempo
    recommendations[user_id] = popularity_last_month.head(10)['movieId'].tolist()

    # Si el usuario no ha interactuado en el último período, llenar las recomendaciones con películas populares generales
    if len(recommendations[user_id]) < 10:
        general_recommendations = popularity_per_month.head(10)['movieId'].tolist()
        recommendations[user_id].extend(general_recommendations[:10-len(recommendations[user_id])])

# Imprimir las recomendaciones para los primeros diez usuarios
for user_id in merged_dfe['userId'].unique()[:10]:
    print("Usuario:", user_id)
    print("Recomendaciones:", recommendations[user_id])
    print()

Usuario: 130712
Recomendaciones: [2012, 187541, 79132, 2571, 318, 122912, 58559, 2959, 7153, 4993]

Usuario: 39332
Recomendaciones: [39, 60069, 55247, 55269, 55280, 56367, 56941, 58559, 60950, 54001]

Usuario: 51898
Recomendaciones: [194448, 79132, 2571, 318, 122912, 58559, 2959, 7153, 4993, 5952]

Usuario: 82695
Recomendaciones: [2019, 128620, 79132, 2571, 318, 122912, 58559, 2959, 7153, 4993]

Usuario: 54901
Recomendaciones: [42725, 52245, 79132, 2571, 318, 122912, 58559, 2959, 7153, 4993]

Usuario: 8495
Recomendaciones: [44195, 166568, 79132, 2571, 318, 122912, 58559, 2959, 7153, 4993]

Usuario: 127446
Recomendaciones: [47, 184931, 184157, 183613, 183611, 183421, 183295, 183155, 183011, 180555]

Usuario: 133240
Recomendaciones: [1262, 1645, 168252, 162598, 145150, 139385, 122922, 122920, 122918, 122916]

Usuario: 70851
Recomendaciones: [6, 5388, 194951, 194947, 193900, 192803, 192245, 181475, 178523, 33681]

Usuario: 109615
Recomendaciones: [192379, 194006, 79132, 2571, 318, 122912,

In [75]:
#Punto 5): Evaluación con MAP.

In [76]:
def mean_average_precision(actual, predicted, k=10):
    average_precision_sum = 0.0
    num_users = len(actual)

    for i in range(num_users):
        user_actual = actual[i]
        user_predicted = predicted[i][:k]

        num_hits = 0
        precision_sum = 0.0

        # Verificar si el usuario tiene películas valoradas en el conjunto de prueba
        if len(user_actual) > 0:
            for j in range(k):
                if user_predicted[j] in user_actual:
                    num_hits += 1
                    precision_sum += num_hits / (j + 1)

            average_precision_sum += precision_sum / min(len(user_actual), k)

    # Verificar si hay usuarios con películas valoradas en el conjunto de prueba
    if num_users > 0:
        return average_precision_sum / num_users
    else:
        return 0.0

# Obtener las películas valoradas por cada usuario en el conjunto de prueba
actual_ratings = []
for user_id in merged_dfe['userId'].unique():
    user_actual = set(testset[testset['userId'] == user_id]['movieId'])
    actual_ratings.append(list(user_actual))

# Calcular el MAP
predicted_recommendations = [recommendations[user_id] for user_id in merged_dfe['userId'].unique()]
map_score = mean_average_precision(actual_ratings, predicted_recommendations)
print("Mean Average Precision (MAP):", map_score)

Mean Average Precision (MAP): 0.07914261012613812
